<a href="https://colab.research.google.com/github/mathengem/Algorithmic-Trading-Backtesting-in-python/blob/main/z3-shamir-secret-sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 10.1 MB/s eta 0:00:00


In [2]:
# Definition of simple functions to encode a string into an integer and back

import binascii

def str_to_int(str):
  byts = str.encode("utf-8")
  return int(binascii.hexlify(byts), 16)

def int_to_str(i):
  hx = hex(i)[2:]
  secret = binascii.unhexlify(hx)
  return secret.decode("utf-8")

In [3]:
import random

# Function that generates another function given a list of coefficient.
# (also determining the polynomial's degree)
# https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing#Usage
def generate_polynomial_fn(a_vars):
  parts = []
  for degree in range(len(a_vars)):
    # closure on variable 'degree' using default parameter
    def polynomial_part(x, d=degree):
      return a_vars[d] * x**d
    parts.append(polynomial_part)

  # Returns a lambda that sums all part of the polynomial for a given 'x'
  return lambda x: sum(map(lambda f: f(x), parts))

In [4]:
def shamir_split(m, n, secret):
  # Choosing random numbers that are not too far away from our secret
  rnds = [int(random.uniform(0.2, 0.8) * secret) for _ in range(m - 1)]
  f = generate_polynomial_fn([secret] + rnds)
  # returns the evaluation of the polynomial function for 0 < x < n+1
  # WARNING: here x=0 would reveal the secret (f(0) == secret)
  return [(x, f(x)) for x in range(1, n + 1)]

In [5]:
from z3 import *

# https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing#Solution
def shamir_resolve(splits):
  degree = len(splits)

  # Z3 way of declaring unknown variables
  a = [Int(f'a{i}') for i in range(0, degree)]
  solver = Solver()

  # Create a simple system of equations from the splits
  for x, y in splits:
    p = generate_polynomial_fn(a)
    solver.add(y == p(x))

  # This part is Z3 specific, we check and solve the system of equations
  solver.check()
  model = solver.model()
  sec_int = model[a[0]].as_long()

  return int_to_str(sec_int)

In [6]:
secret = "this is so secret"

# We encode our secret into an integer
sec_int = str_to_int(secret)

# Make sure our reverse function works as expected
assert(int_to_str(sec_int) == secret)

In [7]:
m = 7
n = 12
splits = shamir_split(m, n, sec_int)

assert(len(splits) == n)
print(splits)

[(1, 148772187351307516223958825888253161399668), (2, 1832287510622872199803291348696175142397300), (3, 13647066194289311204120363767881218219206004), (4, 63449689653549424152499172004375621989983604), (5, 217217532210100922514935980224419165161809268), (6, 604489697253030327772674673511088579138446708), (7, 1450427769386618644050522403294640912750241140), (8, 3114496382565062805216905525541670848126150004), (9, 6135763604214112896451665831703081967707907444), (10, 11284821135339625150281597072420871972406322548), (11, 19622324326623030717083721773993731850899711348), (12, 32564152010503720210056308347601459000074462580)]


In [8]:
# Picking arbitrarily a subset of the splits
split_subset = splits[3:10]

# Make sure the chosen subset is greater or equal to m
assert(len(split_subset) >= m)

# Reconstruct the secret using only the splits (and the 'x' value)
shamir_resolve(split_subset)

'this is so secret'